In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('data.csv')
df = df.drop(['subject_id-2', 'hadm_id-2', 'subject_id-3', 'stay_id-2', 'subject_id-4', 'stay_id-3','glucose_min', 'glucose_max', 'glucose_mean'], axis=1)
df = df.rename(columns={"glucose_min-2": "glucose_min", "glucose_max-2": "glucose_max"})
df.head()

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,ethnicity,...,ck_cpk_min,ck_cpk_max,ck_mb_min,ck_mb_max,ggt_min,ggt_max,ld_ldh_min,ld_ldh_max,mech_vent,vasopressin
0,10018501,28479513,34739077,M,NaN,2141-07-30 22:34:00,2141-08-05 18:06:00,5.813889,83,WHITE,...,124.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,11424857,22010683,33113817,M,NaN,2155-06-03 18:10:00,2155-06-21 12:58:00,17.783333,51,WHITE,...,NaN,NaN,2.0,2.0,NaN,NaN,238.0,281.0,1,0
2,11797249,29600029,34926613,F,NaN,2159-11-02 09:24:00,2159-11-09 15:45:00,7.264583,36,BLACK/AFRICAN AMERICAN,...,32.0,32.0,NaN,NaN,NaN,NaN,153.0,153.0,0,0
3,12196836,21710144,31460719,M,NaN,2121-02-18 02:41:00,2121-02-22 17:15:00,4.606944,56,WHITE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,12242414,26119789,30542485,M,NaN,2140-11-09 00:00:00,2140-11-13 16:45:00,4.697917,73,WHITE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [27]:
outcome = 'hospital_expire_flag'

general_cols = ['subject_id', 'hadm_id', 'stay_id', 'gender', 'dod', 'admittime',
       'dischtime', 'los_hospital', 'admission_age', 'ethnicity',
       'hospstay_seq', 'first_hosp_stay', 'icu_intime',
       'icu_outtime', 'los_icu', 'icustay_seq', 'first_icu_stay']
charlson_cols = ['age_score',
       'myocardial_infarct', 'congestive_heart_failure',
       'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia',
       'chronic_pulmonary_disease', 'rheumatic_disease',
       'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc',
       'diabetes_with_cc', 'paraplegia', 'renal_disease', 'malignant_cancer',
       'severe_liver_disease', 'metastatic_solid_tumor', 'aids',
       'charlson_comorbidity_index']
vitals_cols = ['heart_rate_min', 'heart_rate_max',
       'heart_rate_mean', 'sbp_min', 'sbp_max', 'sbp_mean', 'dbp_min',
       'dbp_max', 'dbp_mean', 'mbp_min', 'mbp_max', 'mbp_mean',
       'resp_rate_min', 'resp_rate_max', 'resp_rate_mean', 'temperature_min',
       'temperature_max', 'temperature_mean', 'spo2_min', 'spo2_max',
       'spo2_mean']
labs_cols = ['hematocrit_min', 'hematocrit_max', 'hemoglobin_min',
       'hemoglobin_max', 'platelets_min', 'platelets_max', 'wbc_min',
       'wbc_max', 'albumin_min', 'albumin_max', 'globulin_min', 'globulin_max',
       'total_protein_min', 'total_protein_max', 'aniongap_min',
       'aniongap_max', 'bicarbonate_min', 'bicarbonate_max', 'bun_min',
       'bun_max', 'calcium_min', 'calcium_max', 'chloride_min', 'chloride_max',
       'creatinine_min', 'creatinine_max', 'glucose_min', 'glucose_max',
       'sodium_min', 'sodium_max', 'potassium_min', 'potassium_max',
       'abs_basophils_min', 'abs_basophils_max', 'abs_eosinophils_min',
       'abs_eosinophils_max', 'abs_lymphocytes_min', 'abs_lymphocytes_max',
       'abs_monocytes_min', 'abs_monocytes_max', 'abs_neutrophils_min',
       'abs_neutrophils_max', 'atyps_min', 'atyps_max', 'bands_min',
       'bands_max', 'imm_granulocytes_min', 'imm_granulocytes_max',
       'metas_min', 'metas_max', 'nrbc_min', 'nrbc_max', 'd_dimer_min',
       'd_dimer_max', 'fibrinogen_min', 'fibrinogen_max', 'thrombin_min',
       'thrombin_max', 'inr_min', 'inr_max', 'pt_min', 'pt_max', 'ptt_min',
       'ptt_max', 'alt_min', 'alt_max', 'alp_min', 'alp_max', 'ast_min',
       'ast_max', 'amylase_min', 'amylase_max', 'bilirubin_total_min',
       'bilirubin_total_max', 'bilirubin_direct_min', 'bilirubin_direct_max',
       'bilirubin_indirect_min', 'bilirubin_indirect_max', 'ck_cpk_min',
       'ck_cpk_max', 'ck_mb_min', 'ck_mb_max', 'ggt_min', 'ggt_max',
       'ld_ldh_min', 'ld_ldh_max']
mv_vaso_cols = ['mech_vent', 'vasopressin']

In [170]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score

In [29]:
df.ethnicity = df.ethnicity.apply(lambda x: "Others" if x in ['UNKNOWN', 'OTHER', 'UNABLE TO OBTAIN', 'AMERICAN INDIAN/ALASKA NATIVE'] else x)
df.ethnicity.value_counts()

WHITE                     33725
Others                     8562
BLACK/AFRICAN AMERICAN     4575
HISPANIC/LATINO            1739
ASIAN                      1447
Name: ethnicity, dtype: int64

In [150]:
X = df[['gender', 'admission_age', 'ethnicity'] + charlson_cols + vitals_cols + labs_cols + mv_vaso_cols]
X = pd.get_dummies(X, columns=['gender', 'ethnicity']).drop(['gender_F', 'ethnicity_WHITE'], axis=1)
X = X.drop(X.columns[X.isna().all()], axis=1)
feature_names = X.columns
y = df[outcome]
imp = SimpleImputer(missing_values=np.nan, strategy='mean', verbose=1)
X = imp.fit_transform(X)

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [156]:
sscaler = StandardScaler().fit(X_train)
X_train = sscaler.transform(X_train)
X_test = sscaler.transform(X_test)
sfm = SelectFromModel(LassoCV(max_iter=10000, cv=5), threshold=-np.inf, max_features=20).fit(X_train, y_train)
selected_features = feature_names[sfm.get_support()]
print(selected_features)
X_train = sfm.transform(X_train)
X_test = sfm.transform(X_test)

Index(['admission_age', 'age_score', 'cerebrovascular_disease',
       'heart_rate_min', 'heart_rate_mean', 'dbp_mean', 'mbp_mean',
       'resp_rate_mean', 'temperature_mean', 'spo2_min', 'hematocrit_min',
       'hemoglobin_max', 'wbc_min', 'aniongap_min', 'bun_min', 'sodium_max',
       'bilirubin_indirect_min', 'mech_vent', 'vasopressin',
       'ethnicity_Others'],
      dtype='object')


In [199]:
# model = LogisticRegression(max_iter=10000)
# model = LogisticRegression(max_iter=10000, penalty='l2', solver='saga')
# model = LogisticRegression(max_iter=10000, penalty='l1', solver='saga')
# model = DecisionTreeClassifier()
# model = LinearSVC(max_iter=10000)
# model = RandomForestClassifier()
model = AdaBoostClassifier()
# model = GradientBoostingClassifier()
model.fit(X_train, y_train)

AdaBoostClassifier()

In [200]:
# Training data performance
pred = model.predict(X_train)
true = y_train
print('Accuracy:', accuracy_score(true, pred))
print('AUROC:', roc_auc_score(true, pred))

Accuracy: 0.9137384751519996
AUROC: 0.6486943967664377


In [201]:
# Test data performance
pred = model.predict(X_test)
true = y_test
print('Accuracy:', accuracy_score(true, pred))
print('AUROC:', roc_auc_score(true, pred))

Accuracy: 0.9106893106893107
AUROC: 0.645442778874374


In [205]:
for i, j in zip(selected_features, model.feature_importances_):
    print(i, j)

admission_age 0.06
age_score 0.0
cerebrovascular_disease 0.02
heart_rate_min 0.06
heart_rate_mean 0.06
dbp_mean 0.0
mbp_mean 0.08
resp_rate_mean 0.1
temperature_mean 0.04
spo2_min 0.1
hematocrit_min 0.0
hemoglobin_max 0.04
wbc_min 0.1
aniongap_min 0.08
bun_min 0.08
sodium_max 0.04
bilirubin_indirect_min 0.02
mech_vent 0.04
vasopressin 0.06
ethnicity_Others 0.02
